In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [9]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [10]:
layers = tf.keras.layers

In [11]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(10, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [12]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [13]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [15]:
#　model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

In [16]:
model.load_weights('./model_weights/vgg16_train_without_DA=all_ce.h5')

In [17]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[1.8049047053813934, 0.7907]

In [18]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce-without-momentum.npy')
model.evaluate(x=tmp[:512], y=y_test_all[:512], verbose=0)

[3.794926792383194, 0.6542969]

In [18]:
time = [1e3, 5e3, 1e4, 2e4, 4e4, 8e4, 16e4]
for i, t in enumerate(time):
    time[i] = str(int(t))
time.append('None')

csv_file_name = "vgg16"
table = onp.zeros((3, 8))

for i, t in enumerate(time):
    
    tmp = onp.load('./batch_NTK_simple_time=%s.npy'%(t))
    result = model.evaluate(tmp, y_test[:512], verbose=0)
    table[0][i] = result[1]
    
    tmp = onp.load('./batch_NTK_simple_no_dense_time=%s.npy'%(t))
    result = model.evaluate(tmp, y_test[:512], verbose=0)
    table[1][i] = result[1]
    
    tmp = onp.load('./batch_NTK_simple_stride_no_dense_time=%s.npy'%(t))
    result = model.evaluate(tmp, y_test[:512], verbose=0)
    table[2][i] = result[1]
    
onp.savetxt(csv_file_name+".csv", table, delimiter=",", fmt='%.3f')

In [18]:
tmp = onp.load('batch_NTK_simple_no_dense.npy')
model.evaluate(x=tmp, y=y_test_all[:2048], verbose=0)

[3.0394137874245644, 0.67578125]

In [19]:
tmp = onp.load('batch_NTK_simple_stride_no_dense.npy')
model.evaluate(x=tmp, y=y_test_all[:2048], verbose=0)

[2.891904989257455, 0.69140625]

In [18]:
csv_file_name = "vgg16_decrease"
table = onp.zeros((4, 8))
lambd = ["0.00", "0.25", "0.50", "0.75"]
for l in range(4):
    lamb = lambd[l]
    for idx, t in enumerate(np.load('time.npy')):
        file = './variance-and-time/decrease/batch_NTK_simple_decrease_variance_lambda=' + lamb + '_time=%d.npy'%(t)
        tmp = onp.load(file)
        # print('evaluating file: %s'%(file))
        result = model.evaluate(tmp, y_test[:512], verbose=0)
        table[l][idx] = result[1]
        # print(result)
onp.savetxt(csv_file_name+".csv", table, delimiter=",", fmt='%.3f')

In [19]:
csv_file_name = "vgg16_increase"
table = onp.zeros((4, 8))
lambd = ["0.00", "0.25", "0.50", "0.75"]
for l in range(4):
    lamb = lambd[l]
    for idx, t in enumerate(np.load('time.npy')):
        file = './variance-and-time/increase/batch_NTK_simple_increase_variance_lambda=' + lamb + '_time=%d.npy'%(t)
        tmp = onp.load(file)
        # print('evaluating file: %s'%(file))
        result = model.evaluate(tmp, y_test[:512], verbose=0)
        table[l][idx] = result[1]
        # print(result)
onp.savetxt(csv_file_name+".csv", table, delimiter=",", fmt='%.3f')

In [18]:
csv_file_name = "vgg16_decrease"
table = onp.zeros((8, 4))
for l in range(8):
    lamb = 10**l
    for idx, t in enumerate([1e5, 5e5, 1e6, 2.3e6]):
        file = './batch_NTK_simple_increase_variance_lambda=%d_time=%d.npy'%(lamb, t)
        tmp = onp.load(file)
        # print('evaluating file: %s'%(file))
        result = model.evaluate(tmp, y_test[:512], verbose=0)
        table[l][idx] = result[1]
        # print(result)
onp.savetxt(csv_file_name+".csv", table, delimiter=",", fmt='%.3f')

In [18]:
# with pooling, acc can reach 72.2%
# w.o. pooling, acc is at most 67%

In [17]:
file_list = ['./variance-and-time/cifar-fgsm-eps-0.03-time-500000.npy',
             './variance-and-time/cifar-fgsm-eps-0.03-time-100000.npy',
             './variance-and-time/cifar-fgsm-eps-0.03-time-1000000.npy',
             './variance-and-time/cifar-fgsm-eps-0.03-time-2300000.npy',
             './npy/batch_NTK_simple.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy',
             './npy/batch_NTK_cnn19.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy']

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp[:512], y_test[:512], verbose=0)
    print(result)

evaluating file: ./variance-and-time/cifar-fgsm-eps-0.03-time-500000.npy
[3.012692492455244, 0.7011719]
evaluating file: ./variance-and-time/cifar-fgsm-eps-0.03-time-100000.npy
[3.013619028031826, 0.7011719]
evaluating file: ./variance-and-time/cifar-fgsm-eps-0.03-time-1000000.npy
[3.012692492455244, 0.7011719]
evaluating file: ./variance-and-time/cifar-fgsm-eps-0.03-time-2300000.npy
[3.012692492455244, 0.7011719]
evaluating file: ./npy/batch_NTK_simple.npy
[3.012692492455244, 0.7011719]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy
[3.7776207253336906, 0.6542969]
evaluating file: ./npy/batch_NTK_cnn19.npy
[2.7299178019165993, 0.72265625]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy
[3.0676044151186943, 0.70703125]


In [19]:
file_list = ['cifar-fgsm-eps-0.03-time-500000.npy',
             'cifar-fgsm-eps-0.03-time-100000.npy',
             'cifar-fgsm-eps-0.03-time-1000000.npy',
             'cifar-fgsm-eps-0.03-time-2300000.npy',
             './npy/batch_NTK_simple.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy',
             './npy/batch_NTK_cnn19.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy',
             "./npy/batch_NTK_cnn19_decrease_variance.npy",
             "./npy/batch_NTK_cnn19_increase_variance.npy",
             "./npy/batch_NTK_simple_decrease_variance.npy",
             "./npy/batch_NTK_simple_increase_variance.npy"]

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp[:512], y_test[:512], verbose=0)
    print(result)

evaluating file: cifar-fgsm-eps-0.03-time-500000.npy
[3.012692492455244, 0.7011719]
evaluating file: cifar-fgsm-eps-0.03-time-100000.npy
[3.013619028031826, 0.7011719]
evaluating file: cifar-fgsm-eps-0.03-time-1000000.npy
[3.012692492455244, 0.7011719]
evaluating file: cifar-fgsm-eps-0.03-time-2300000.npy
[3.012692492455244, 0.7011719]
evaluating file: ./npy/batch_NTK_simple.npy
[3.012692492455244, 0.7011719]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy
[3.7776207253336906, 0.6542969]
evaluating file: ./npy/batch_NTK_cnn19.npy
[2.7299178019165993, 0.72265625]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy
[3.0676044151186943, 0.70703125]
evaluating file: ./npy/batch_NTK_cnn19_decrease_variance.npy
[2.7365561313927174, 0.7246094]
evaluating file: ./npy/batch_NTK_cnn19_increase_variance.npy
[2.7307813093066216, 0.7207031]
evaluating file: ./npy/batch_NTK_simple_decrease_variance.npy
[2.999278411269188, 0.703125]
evaluating f

In [21]:
file_list = ['batch_NTK_simple_decrease_variance.npy',
             'batch_NTK_simple_increase_variance.npy',
             'batch_NTK_CNN10_decrease_variance.npy',
             'batch_NTK_CNN10_increase_variance.npy']

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp, y_test[:2048], verbose=0)
    print(result)

evaluating file: batch_NTK_simple_decrease_variance.npy
[2.8779119504615664, 0.69384766]
evaluating file: batch_NTK_simple_increase_variance.npy
[2.8760974910110235, 0.68652344]
evaluating file: batch_NTK_CNN10_decrease_variance.npy
[2.670802805572748, 0.71191406]
evaluating file: batch_NTK_CNN10_increase_variance.npy
[2.664850667119026, 0.7084961]


In [18]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[1.726710021495819, 0.78125]

In [19]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[1.3806814041454345, 0.8310547]

In [20]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========MSE============


[1.373602248262614, 0.84033203]

In [22]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[1.6405316591262817, 0.8046875]

In [19]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[2.665903329849243, 0.708984375]

In [20]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[2.8858518600463867, 0.68896484375]

In [21]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[2.7764017581939697, 0.71240234375]

In [22]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========small============


[3.704181671142578, 0.65087890625]